**Submitted by `your_name` on `date_of_submission`**

# Optimization Exercises

This notebook was written by Selin Ataç (selin.atac@epfl.ch) and edited by Dr. Léa Ricard (lea.ricard@epfl.ch) for the Optimization and Simulation course at EPFL (https://edu.epfl.ch/studyplan/en/doctoral_school/civil-and-environmental-engineering/coursebook/optimization-and-simulation-MATH-600). 

Please contact before distributing or reusing the material below.


## Table of Contents
* [Travelling Salesman Problem](#Travelling-Salesman-Problem)
    * [Problem definition and encoding](#Problem-definition)
    * [Implementation: The core functionality](#Implementation)
* [Optimization Algorithms](#Optimization-algorithms)
    * [Exercise 1: Full enumeration](#Exercise-1:-Full-enumeration)
    * [Exercise 2: Greedy algorithm](#Exercise-2:-Greedy-algorithm)
    * [Exercise 3: Local search](#Exercise-3:-Local-search)
    * [Exercise 4: Variable Neighborhood Search](#Exercise-4:-Variable-Neighborhood-Search)
    * [Exercise 5: Simulated annealing](#Exercise-5:-Simulated-annealing)
* [Testing the optimization algorithms and solution profiling](#Testing-the-optimization-algorithms-and-solution-profiling)

## Travelling Salesman Problem

### Problem definition

- A salesman must visit $n$ cities.
- Every city must be visited exactly once.
- The salesman starts and ends the trip at their home city.
- The total trip length is assumed to be the cost of the travel.

### Objective

What sequence of cities minimizes the travel cost?

### Problem encoding

We consecutively number the cities: $0,..., n$

We encode the solutions as $x=(x_0, x_1,..., x_n, x_0)$ where

- $x_0$ is the index of the home city,
- $x_i$ is the index of the $i^{th}$ city visited along the way, and
- $x_n$ is the index of the last city visited before returning home.

### Implementation

#### The required python libraries
You will use the following python libraries in this exercise: `numpy`, `plotly`. Install it using `pip` on your command line:

    pip install numpy plotly

or if you are using conda:

    conda install numpy
    conda install -c plotly plotly

In [2]:
import numpy as np
import pandas as pd
import plotly
import plotly.graph_objects as go
import time
import timeit
import math

from numpy.random import Generator, PCG64 
from plotly.subplots import make_subplots

### The core functionality

Before we optimize the path of the salesman, we need to construct several functions to help us simulate and display the path of the salesman.

Begin by implementing a function for simulating city locations on an x-y grid. 
At the end of this step, you will be able to generate a plot of cities on an x-y coordinate plot, path of the salesman, and a list of tuples representing the location of the cities.

1. Implement a function `simulate_cities(rg, n_cities)` which takes as input: 
  - `rg` : a numpy random generator object with a specified seed value
  - `n_cities`: an integer for the number of cities to generate
 
 The return of the function should be a dictionary of tuples, e.g., `{0: (x0,y0), 1: (x1,y1), ..., n: (xn,yn)}`

In [3]:
def simulate_cities(rg:np.random.Generator, n_cities):
    """Function to implement

    Args:
        rg (Generator object): a numpy random generator object with a specified seed value
        n_cities (int) : an integer for the number of cities to generate
        
    Returns:
        cities (dict): a scalar representing the objective function value of the total
            distance travelled
    
    Example:
        rg = Generator(PCG64(42069))
        simulate_cities(rg, 2)
        >>> {0: (2.4, 1.4), 1: (0.2, 3.5)}   
    """  
    # Implement your solution here
    cities = {}
    # Generate random coordinates for each city
    for i in range(n_cities):
        x = rg.uniform(0, 1)
        y = rg.uniform(0, 1)
        cities[i] = (x, y)
    
    return cities

#### Test the function `simulate_cities(rg, n_cities)`

In [4]:
rg = Generator(PCG64(42069)) # set your own unique seed number
simulate_cities(rg, 10)

{0: (0.9478996465627648, 0.2974102502883651),
 1: (0.07180247229372805, 0.3295081953938601),
 2: (0.5594414353429973, 0.8634674736423997),
 3: (0.721347982465723, 0.691424444294593),
 4: (0.5408533259842724, 0.4535449329926894),
 5: (0.8072731499300757, 0.35227676836708866),
 6: (0.6985951690873387, 0.735711881881111),
 7: (0.7701562520511277, 0.7905928224207733),
 8: (0.3989059418598707, 0.34322732737709305),
 9: (0.5754637086577505, 0.6388855586960542)}

2. Implement a function `draw_salesman(path, cities)` which takes as input:
  - `path`: a list of integers which represents the sequence of cities visited, e.g. `[0,1,3,2,4,0]` 
  - `cities`: a dictionary of city x-y coordinates, keyed by the number of the city, use the return value of the `simulate_cities()` function as example: `{0: (x0,y0), 1: (x1,y1), ..., n: (xn,yn)}`
  
 The return of the function should display a visualization of the cities visited.

In [5]:
def draw_salesman(path, cities, title="Path taken"):
    """Function to implement

    Args:
        path (list): a row vector representing the solution to be evaluated
        cities (dict): data structure that contains supplementary 
            information about the problem, in particular the xy-coordinates of the 
            cities.
        **kwargs: arbitrary keyword arguments (optional variables)
    """  
    # loop through the coordinates using the path sequence
    x = [cities[p][0] for p in path]
    y = [cities[p][1] for p in path]

    fig = go.Figure()
    #fig.add_trace(go.Scatter(x=x, y=y, name='path', mode='lines'))
    fig.add_trace(go.Scatter(x=x, y=y, name="cities", mode='markers+text', 
        marker={'size': 10}, text=[str(i) for i in path], textposition="bottom center",))
    
    # plot the path
    for i in range(len(path)-1):
        fig.add_trace(go.Scatter(x=[x[i], x[i+1]], y=[y[i], y[i+1]], mode='lines', line=dict(color='blue', width=2)))
        
    fig.update_layout(template="presentation", title=title, xaxis_title="X", yaxis_title="Y", width=800, height=600)
    fig.update_xaxes(range=[0, 1])
    fig.update_yaxes(range=[0, 1])
    fig.show() # to reveal the figure
    

#### Test the function `draw_salesman(path, cities)`

In [6]:
rg = Generator(PCG64(42069)) # set your own unique seed number
n_cities = 10
cities = simulate_cities(rg, n_cities) # simulate the list of cities
print(cities)
solution = list(range(0, n_cities)) # sample solution of the path of the salesman
solution.append(solution[0])
draw_salesman(solution, cities) # draw salesman

{0: (0.9478996465627648, 0.2974102502883651), 1: (0.07180247229372805, 0.3295081953938601), 2: (0.5594414353429973, 0.8634674736423997), 3: (0.721347982465723, 0.691424444294593), 4: (0.5408533259842724, 0.4535449329926894), 5: (0.8072731499300757, 0.35227676836708866), 6: (0.6985951690873387, 0.735711881881111), 7: (0.7701562520511277, 0.7905928224207733), 8: (0.3989059418598707, 0.34322732737709305), 9: (0.5754637086577505, 0.6388855586960542)}


3. Implement a function `evaluate_city_sequence(path, cities)` which takes as input:
  - `path`: a list of integers which represents the sequence of cities visited, e.g. `[0,1,3,2,4,0]` 
  - `cities`: the dict of cities {n: (x, y)}
  - The return of the function should be the **total length of the travelled path**.
  
We use the Euclidean distance to calculate the distance between each city, including the distance from the final visited city back to the home city.

In [7]:
def evaluate_city_sequence(path, cities=None, **kwargs):
    """Function to implement

    Returns the objective function value of the solution x.

    Args:
        x (list): a row vector representing the solution to be evaluated
        cities (dict, optional): data structure that contains supplementary 
            information about the problem, in particular the xy-coordinates of the 
            cities.
        **kwargs: arbitrary keyword arguments (optional variables)
    
    Returns:
        Q (float): a scalar representing the objective function value of the total
            distance travelled
    
    Example:
        cities = {0: (2.4, 1.4), 1: (0.2, 3.5)}
        calculated = evaluate_city_sequence([0, 1], cities)
        print(calculated)
        >>> 3.041381
    """  
   
    # Implement your solution here
    Q = 0
    for i in range(len(path)-1):
        x1, y1 = cities[path[i]]
        x2, y2 = cities[path[i+1]]
        Q += np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        
    # Add distance from last city to first city
    x1, y1 = cities[path[-1]]
    x2, y2 = cities[path[0]]
    Q += np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

    # Return the total distance
    return Q
    

**Test the function `evaluate_city_sequence(x, cities=None, **kwargs)`** and verify that the calculation is correct.

In [8]:
def evaluate_city_sequence_test(cities={0: (0, 0), 1: (1, 1)}):
    """Very simple test
    
    Salesman starts at city zero with coordinates(0, 0), travels to city one with 
    coordinates(1, 1), and then returns to city zero.
    """

    # expected distance travelled
    expected = 2 * np.sqrt(2)
    
    # call your objective function
    calculated = evaluate_city_sequence([0, 1, 0], cities)
    
    # show your results
    results = print(
        'Expected={0:.3f}, Calculated={1:.3f}'.format(expected, calculated)
    )
    if abs(expected-calculated) < 1e-6:
        print('OK')
    else:
        print('NOT CORRECT')

evaluate_city_sequence_test()

Expected=2.828, Calculated=2.828
OK


## Optimization algorithms

### Optimize traveling salesman path

We want to optimize the path taken by a travelling salesman. 
First, implement a function `randomly_generate_new_city_seq()` that randomly generates a path sequence. This funtion will serve as a benchmark to test "smarter" algorithms.
We assume that the salesman always starts and ends in city `0`.

A very simple example is a full (random) enumeration of the cities.

In [9]:
def randomly_generate_new_city_seq(rg:np.random.Generator, path, cities=None, **kwargs):
    """Function to implement

    Returns a permutation of the row vector "path", where the first and last elements
    of "path" stay unchanged. Implement different specifications. For example:
    - exchanges two randomly selected entries of "path"

    Args:
        rg (Generator object) : a numpy random seed value
        path (list): a row vector representing the current city sequence
        cities (dict, optional): data structure that contains supplementary 
            information about the problem, in particular the coordinates of the 
            cities.
        **kwargs: arbitrary keyword arguments (optional variables)
    
    Returns:
        new_path (list): a row vector with a permutation of "path" 
        
        Where the first and last elements of "new_path" are the first and last elements of "path", respectively
    
    Example:
        path = [0, 1, 2, 3, 4, 5]
        new_path = randomly_generate_new_city_seq(path)
        print(new_path)
        >>> [0, 1, 3, 2, 4, 5]
    """   
    new_path = np.array(path.copy()) # make a copy of path
    
    # Implement your solution here

    # Randomly select two indices to swap
    idx1 = rg.integers(1, len(path) - 3) + 1
    idx2 = rg.integers(1, len(path) - 3) + 1
    while idx1 == idx2:
        idx2 = rg.integers(1, len(path) - 3) + 1

    # Swap the two indices
    new_path[idx1], new_path[idx2] = new_path[idx2], new_path[idx1]
    
    return list(new_path)
    

#### Test the function

In [10]:
rg = Generator(PCG64(42069)) # set your own unique seed number
n_cities = 20
cities = simulate_cities(rg, n_cities)
print(cities)
path = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,0]
print(path)
rg = Generator(PCG64(42070))
path = randomly_generate_new_city_seq(rg, path)
print(path)
draw_salesman(path, cities)

{0: (0.9478996465627648, 0.2974102502883651), 1: (0.07180247229372805, 0.3295081953938601), 2: (0.5594414353429973, 0.8634674736423997), 3: (0.721347982465723, 0.691424444294593), 4: (0.5408533259842724, 0.4535449329926894), 5: (0.8072731499300757, 0.35227676836708866), 6: (0.6985951690873387, 0.735711881881111), 7: (0.7701562520511277, 0.7905928224207733), 8: (0.3989059418598707, 0.34322732737709305), 9: (0.5754637086577505, 0.6388855586960542), 10: (0.6927275748222806, 0.5741830212716611), 11: (0.9892790910022541, 0.8575032696385994), 12: (0.8563625706954108, 0.1541154374490148), 13: (0.826842816994146, 0.012500914270159869), 14: (0.5864646240283783, 0.673065924331924), 15: (0.24567092749107788, 0.7694721151999951), 16: (0.09844735428552698, 0.7003168541007362), 17: (0.7077374270492284, 0.41338568507057805), 18: (0.23973496457856236, 0.46793620274615), 19: (0.06035837036419933, 0.8842708897557718)}
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 0]
[np.int64(0)

### Algorithms

Notice in our sample solution, it was not ideal and the salesman's path criss-crossed. Our goal is to minimize the total distance travelled by the salesman, while visiting all the cities.

We can use the total path travelled as our **objective function**. Recall the function `evaluate_city_sequence()`, this is our objective function. 

Now let us implement a **full enumeration** algorithm. This is a simple trial and error method.

### Exercise 1: Full enumeration

Functions to implement:

`full_enumeration()`

Calculate the computational time and limitations of the full enumeration.

What is the maximum problem size (number of cities) that you could solve with this approach?

In [11]:
from itertools import permutations

In [12]:
def full_enumeration(cities, **kwargs):
    """Function to implement

    Returns the best solution minimizing distance travelled.

    Args:
        cities (dict, optional): data structure that contains supplementary 
            information about the problem, in particular the xy-coordinates of the 
            cities.
        **kwargs: arbitrary keyword arguments (optional variables)
    
    Returns:
        best_path (list): a row vector representing the minimum cost city sequence
        best_path_cost (float): a scalar representing the objective function value of the best path
    
    """  
    
    # implement your own solution here
    best_path = None
    best_path_cost = float('inf')

    # Generate all permutations of the cities
    for path in permutations(range(len(cities))):
        # Calculate the cost of the current path
        path_cost = evaluate_city_sequence(path, cities)
        
        # Update the best path if the current one is better
        if path_cost < best_path_cost:
            best_path_cost = path_cost
            best_path = path

    return best_path, best_path_cost

In [13]:
# Test
rg = Generator(PCG64(42069)) # set your own unique seed number
n_cities = 10
cities = simulate_cities(rg, n_cities) # simulate the list of cities
start_time = time.time()  # Record start time
solution, distance = full_enumeration(cities)
end_time = time.time()  # Record end time
print("best solution:", solution)
print("distance travelled:", distance)
print("Computing time:", end_time - start_time, "seconds")

draw_salesman(solution, cities)

best solution: (1, 9, 2, 7, 6, 3, 0, 5, 4, 8)
distance travelled: 2.5768143746500534
Computing time: 40.7247850894928 seconds


### Exercise 2: Greedy algorithm

**Principles:**
- Each step is taken as decisions from the previous step, therefore depends on the previous step.
- It is easy to implement, but may generate poor solutions.

**The algorithm:**
1. Start from home city
2. Select the next closest city
3. Repeat 2 until all cities have been visited
4. Return to home city

Implement this algorithm in your own Python notebook and test it out. Try to reuse portions of the code in the first exercise.

The function to implement:

`greedy()`

Calculate the computational time.

Profile your greedy algorithm and compare it with the full enumeration algorithm. What do you observe? explain the pros and cons of each method.

In [14]:
def greedy(cities, **kwargs):
    """Function to implement

    Returns a good solution that minimizes distance travelled.

    Args:
        cities (dict, optional): data structure that contains supplementary 
            information about the problem, in particular the xy-coordinates of the 
            cities.
        **kwargs: arbitrary keyword arguments (optional variables)
    
    Returns:
        path (list): a row vector representing a feasible city sequence
        path_cost (float): a scalar representing the objective function value of the path outputted
    
    """ 
    
    # implement your own solution here
    n_cities = len(cities)
    path = [0]  # Start from the first city
    visited = set(path)
    path_cost = 0
    current_city = 0
    next_city = None
    for i in range(1, n_cities):
        min_distance = float('inf')
        next_city = None
        for city in range(n_cities):
            if city not in visited:
                distance = np.sqrt((cities[current_city][0] - cities[city][0])**2 + (cities[current_city][1] - cities[city][1])**2)
                if distance < min_distance:
                    min_distance = distance
                    next_city = city
        path.append(next_city)
        visited.add(next_city)
        path_cost += min_distance
        current_city = next_city

    path_cost = evaluate_city_sequence(path, cities)
    
    return path, path_cost
    

In [15]:
# Test
rg = Generator(PCG64(42069)) # set your own unique seed number
n_cities = 10
cities = simulate_cities(rg, n_cities)
start_time = time.time()  # Record start time
solution, distance = greedy(cities) # new input parameter K
end_time = time.time()  # Record end time
print("best solution:", solution)
print("distance travelled:", distance)
print("Computing time:", end_time - start_time, "seconds")

draw_salesman(solution, cities)

best solution: [0, 5, 4, 8, 1, 9, 3, 6, 7, 2]
distance travelled: 2.738743322903475
Computing time: 0.00014472007751464844 seconds


### Exercise 3: Local search

A local search algorithm iteratively explores neighboring solutions to improve upon an initial candidate, aiming to find the optimal solution by making incremental changes.

**Input**: 

$V(x)$: Neighbourhood structure, where $V(x)$ is the set of feasible neighbors of $x$. Use the 2-OPT neighborhod.

**Initialization**: 
- $x_0$: use the outcome of the greedy algorithm

**Iterations**: 
- At each iteration $k$, consider the neighbors in $V(x_k)$ one at a time
- For each $y \in V(x_k)$, if $f(y) \leq f(x_k)$, then $x_{k+1} = y$
- If $f(y) > f(x_k), \forall y \in V(x_k)$, $x_k$ is a local minimum. Stop.

Functions to implement:

`twoOPT_neighborhood()`

`local_search()`

Calculate the computational time.

In [16]:
def twoOPT_neighborhood(x, cities=None, **kwargs):
    """Function to implement

    Returns the 2-OPT neighborhoods of a path. 

    Args:
        x : a row vector representing the current city sequence
        cities (dict, optional): data structure that contains supplementary 
            information about the problem, in particular the xy-coordinates of the 
            cities.
        **kwargs: arbitrary keyword arguments (optional variables)
    
    Returns:
        neighbors (list): a matrix representing all paths part of the neighbohood of x
    
    """
    
    # implement your own solution here
    n = len(x)
    neighbors = []
    # Generate all 2-opt neighbors
    for i in range(1, n-2):
        for j in range(i+1, n-1):
            # Create a new path by reversing the segment between i and j
            new_path = x[:i] + x[i:j+1][::-1] + x[j+1:]
            neighbors.append(new_path)
    
    return neighbors
    
    

def local_search(cities, **kwargs):
    """Function to implement

     Returns a local minimum of the traveling salesman problem

    Args:
        cities (dict, optional): data structure that contains supplementary 
            information about the problem, in particular the xy-coordinates of the 
            cities.
        **kwargs: arbitrary keyword arguments (optional variables)
    
    Returns:
        x:  a row vector representing a local minimum city sequence
        objValue_x: a scalar representing the objective function value of the path outputted
    
    """
    
    # implement your own solution here
    # Initialize the path using a greedy solution
    x, objValue_x = greedy(cities)
    improved = True
    while improved:
        improved = False
        # Generate 2-opt neighbors
        neighbors = twoOPT_neighborhood(x, cities)
        for neighbor in neighbors:
            # Evaluate the objective function value of the neighbor
            objValue_neighbor = evaluate_city_sequence(neighbor, cities)
            # If the neighbor is better, update x and objValue_x
            if objValue_neighbor < objValue_x:
                x = neighbor
                objValue_x = objValue_neighbor
                improved = True
                break  # Exit the inner loop to start again with the new x

    # Return the best path and its cost
    return x, objValue_x
    

#### Test the function

In [17]:
# Test
rg = Generator(PCG64(42069)) # set your own unique seed number
n_cities = 10
cities = simulate_cities(rg, n_cities)
start_time = time.time()  # Record start time
solution, distance = local_search(cities, iterations=1000)
end_time = time.time()  # Record end time
print("best solution:", solution)
print("distance travelled:", distance)
print("Computing time:", end_time - start_time, "seconds")

draw_salesman(solution, cities)

best solution: [0, 5, 4, 8, 1, 9, 3, 6, 7, 2]
distance travelled: 2.738743322903475
Computing time: 0.0018947124481201172 seconds


### Exercise 4: Variable Neighborhood Search

In variable neighbourhood search, we consider several neighborhood structures.

When a local optimum has been found for a given neighborhood structure, continue with another structure.

**Input**: 

$V_1, V_2, ..., V_K$: Neighborhood structures
where $K$ is the total number of neighborhoods

**Initialization**: 
- $x_c \leftarrow x_0$ (initial solution)
- $k\leftarrow 0$ 

Functions to implement:

`neighborhood_1()`

`neighborhood_2()`

`neighborhood_...()`

`neighborhood_k()`

`variable_neighborhood_search()`

Calculate the computational time.

In [18]:
# Test
import itertools
l = [1, 2, 3]
combs = itertools.combinations(l, 2)
print(list(combs))

[(1, 2), (1, 3), (2, 3)]


In [19]:
def neighborhood_1(x, cities=None, **kwargs):
    """Function to implement

    This function returns the neighborhoods of a path. The neighbourhood is defined as swapping t

    Args:
        x : a row vector representing the current city sequence
        cities (dict, optional): data structure that contains supplementary 
            information about the problem, in particular the xy-coordinates of the 
            cities.
        **kwargs: arbitrary keyword arguments (optional variables)
    
    Returns:
        neighbors (list): a matrix representing all paths part of the neighbohood of x
    """
    x = np.array(x)
    
    # implement your own solution here
    neighbors = []
    for i, j in itertools.combinations(np.arange(len(x)), 2):

        # Swap the two cities
        new_path = x.copy()
        new_path[i], new_path[j] = new_path[j], new_path[i]
        neighbors.append(new_path)
    
    return neighbors

def neighborhood_2(x, cities=None, **kwargs):
    """Function to implement

    This function returns the neighborhoods of a path. The neighbourhood is defined as swapping t

    Args:
        x : a row vector representing the current city sequence
        cities (dict, optional): data structure that contains supplementary 
            information about the problem, in particular the xy-coordinates of the 
            cities.
        **kwargs: arbitrary keyword arguments (optional variables)
    
    Returns:
        neighbors (list): a matrix representing all paths part of the neighbohood of x
    """
    x = np.array(x)
    
    # implement your own solution here
    neighbors = []
    for i, j, k in itertools.combinations(np.arange(len(x)), 3):

        for i_, j_, k_ in itertools.permutations([i, j, k]):
            if i_ == i and j_ == j and k_ == k:
                continue
            
            # Swap the two cities
            new_path = x.copy()
            new_path[i_], new_path[j_], new_path[k_] = new_path[i], new_path[j], new_path[k]
            neighbors.append(new_path)

    
    return neighbors
    

def variable_neighborhood_search(cities, iterations=600, **kwargs):
    """Function to implement

     Returns a local minimum of the traveling salesman problem

    Args:
        cities (dict, optional): data structure that contains supplementary 
            information about the problem, in particular the xy-coordinates of the 
            cities.
        iterations (int): maximum number of iterations of the algorithm
        **kwargs: arbitrary keyword arguments (optional variables)
    
    Returns:
        x:  a row vector representing a local minimum city sequence
        objValue_x: a scalar representing the objective function value of the path outputted
    
    """
    
    # implement your own solution here

    # Initialize the path using a greedy solution
    x, objValue_x = greedy(cities)
    print(f"greedy x: {x}")

    improved = True
    iter = 0
    while iter < iterations:
        improved = False

        # Generate 2-opt neighbors
        neighbors = neighborhood_1(x, cities)
        for neighbor in neighbors:
            iter += 1

            # Evaluate the objective function value of the neighbor
            objValue_neighbor = evaluate_city_sequence(neighbor, cities)

            # If the neighbor is better, update x and objValue_x
            if objValue_neighbor < objValue_x:
                x = neighbor
                objValue_x = objValue_neighbor
                improved = True
                print(f"neighborhood_1: x = {x}")
                break  # Exit the inner loop to start again with the new x

        if improved:
            continue

        neighbors = neighborhood_2(x, cities)
        for neighbor in neighbors:
            iter += 1

            # Evaluate the objective function value of the neighbor
            objValue_neighbor = evaluate_city_sequence(neighbor, cities)

            # If the neighbor is better, update x and objValue_x
            if objValue_neighbor < objValue_x:
                x = neighbor
                objValue_x = objValue_neighbor
                improved = True
                print(f"neighborhood_2: x = {x}")
                break

    # Return the best path and its cost          
    return x, objValue_x
    

#### Test the function

In [20]:
# Test
rg = Generator(PCG64(46)) # set your own unique seed number
n_cities = 10
cities = simulate_cities(rg, n_cities)
start_time = time.time()  # Record start time
solution, distance = variable_neighborhood_search(cities, iterations=100)
end_time = time.time()  # Record end time
print("best solution:", solution)
print("distance travelled:", distance)
print("Computing time:", end_time - start_time, "seconds")

draw_salesman(solution, cities)

greedy x: [0, 2, 6, 5, 7, 3, 4, 1, 9, 8]
neighborhood_1: x = [2 0 6 5 7 3 4 1 9 8]
neighborhood_1: x = [6 0 2 5 7 3 4 1 9 8]
neighborhood_1: x = [6 0 2 5 7 3 4 1 8 9]
best solution: [6 0 2 5 7 3 4 1 8 9]
distance travelled: 2.7999657632682218
Computing time: 0.013420581817626953 seconds


### Exercise 5: Simulated annealing

In simulated annealing, we consider solutions that are not better from the current best solution with a probability.

***
- Select a random solution $y \in V(x_k)$
- If $f(y) \leq f(x_k)$ 
    - $x_{k+1} = y$
- Else 
    - $x_{k+1} = y$ with probability $e^{-\frac{f(y)-f(x_k)}{T}}$ with T > 0
***

To deal with that, one can draw $r$ from $uniform(0,1)$ distribution and accept $y$ if $e^{-\frac{f(y)-f(x_k)}{T}} > r$.

Functions to implement:

`temperature_upt()`

`simulated_annealing()`

Calculate the computational time.

In [55]:
# def temperature_upt(max_t_changes, t_changes, avg_inc_obj, accep_init, accep_final):
    
#     # implement your own solution here
    
#     return temperature

def simulated_annealing(cities, iterations=600, **kwargs):
    """Function to implement

     Returns a good feasible solution to the traveling salesman problem

    Args:
        cities (dict, optional): data structure that contains supplementary 
            information about the problem, in particular the xy-coordinates of the 
            cities.
        iterations (int): maximum number of iterations of the algorithm
        **kwargs: arbitrary keyword arguments (optional variables)
    
    Returns:
        x:  a row vector representing a local minimum city sequence
        objValue_x: a scalar representing the objective function value of the path outputted
    """

    # implement your own solution here
    rg = Generator(PCG64(46))
    temperature = 0.01

    # Initialize the path using a greedy solution
    x, objValue_x = greedy(cities)
    print(f"greedy x: {x}, distance: {objValue_x}")

    worse_accept_count = 0
    for iter in range(iterations):

        # Generate a random path
        path = rg.permutation(len(cities))
        objValue_path = evaluate_city_sequence(path, cities)

        # # swap two cities in the path
        # path = x.copy()
        # idx1 = rg.integers(1, len(path) - 2) + 1
        # idx2 = rg.integers(1, len(path) - 2) + 1    
        # while idx1 == idx2:
        #     idx2 = rg.integers(1, len(path) - 2) + 1
        # path[idx1], path[idx2] = path[idx2], path[idx1]
        # objValue_path = evaluate_city_sequence(path, cities)

        # # Generate a random path
        # path = rg.permutation(len(cities))
        # objValue_path = evaluate_city_sequence(path, cities)
        if objValue_path < objValue_x:
            x = path
            objValue_x = objValue_path
            print(f"iter: {iter}, better path found: {x}, distance: {objValue_x}")
        else:
            # Calculate the acceptance probability
            t_prob = np.exp((objValue_x - objValue_path) / temperature)
            ran_var = rg.uniform(0, 1)
            if ran_var < t_prob:
                x = path
                objValue_x = objValue_path
                print(f"iter: {iter}, worse path accepted: {x}, distance: {objValue_x}")
                worse_accept_count += 1
    
    print(f"worse_accept_count: {worse_accept_count}")
    return x, objValue_x
        
    

#### Test the function

In [58]:
# Test
rg = Generator(PCG64(42069)) # set your own unique seed number
n_cities = 10
cities = simulate_cities(rg, n_cities)
start_time = time.time()  # Record start time
solution, distance = simulated_annealing(cities, iterations = 600)
end_time = time.time()  # Record end time
print("best solution:", solution)
print("distance travelled:", distance)
print("Computing time:", end_time - start_time, "seconds")

draw_salesman(solution, cities)

greedy x: [0, 5, 4, 8, 1, 9, 3, 6, 7, 2], distance: 2.738743322903475
iter: 121, better path found: [7 3 6 5 0 4 8 1 9 2], distance: 2.6921437884971797
worse_accept_count: 0
best solution: [7 3 6 5 0 4 8 1 9 2]
distance travelled: 2.6921437884971797
Computing time: 0.06993603706359863 seconds


## Testing the optimization algorithms and solution profiling

### Testing
#### Test your optimization functions using `optimization_TSP_test()`

In [59]:
# Run the optimization by reusing the functions simulateCities(), drawSalesman(), evaluate()

def optimization_TSP_test(rg, n_cities=15, iterations=600):
    cities = simulate_cities(rg, n_cities)
    inital_solution = list(range(len(cities))) + [0]
    print(cities)
    print("initial solution:", inital_solution)
    print("inital distance travelled:", evaluate_city_sequence(inital_solution, cities))
    draw_salesman(inital_solution, cities, "Initial Solution") # show the inital solution
    
    algorithms = {
    'Full Enumeration': full_enumeration,
    'Greedy algorithm': greedy,
    'Local search algorithm': local_search,
    'Variable neighborhood search algorithm': variable_neighborhood_search,
    'Simulated annealing algorithm': simulated_annealing
    }
    
    for algorithm_name, algorithm in algorithms.items():
        print(algorithm_name + "\n-----------")
        start_time = time.time()  # Record start time
        solution, distance = algorithm(cities, iterations=iterations)
        end_time = time.time()  # Record end time
        print("best solution:", solution)
        print("distance travelled:", distance)
        print("Computing time:", end_time - start_time, "seconds")
        draw_salesman(solution, cities, algorithm_name)

# run all
rg = Generator(PCG64(42069)) 
optimization_TSP_test(rg, n_cities=10, iterations=600)

{0: (0.9478996465627648, 0.2974102502883651), 1: (0.07180247229372805, 0.3295081953938601), 2: (0.5594414353429973, 0.8634674736423997), 3: (0.721347982465723, 0.691424444294593), 4: (0.5408533259842724, 0.4535449329926894), 5: (0.8072731499300757, 0.35227676836708866), 6: (0.6985951690873387, 0.735711881881111), 7: (0.7701562520511277, 0.7905928224207733), 8: (0.3989059418598707, 0.34322732737709305), 9: (0.5754637086577505, 0.6388855586960542)}
initial solution: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0]
inital distance travelled: 4.33939218205926


Full Enumeration
-----------
best solution: (1, 9, 2, 7, 6, 3, 0, 5, 4, 8)
distance travelled: 2.5768143746500534
Computing time: 39.63637566566467 seconds


Greedy algorithm
-----------
best solution: [0, 5, 4, 8, 1, 9, 3, 6, 7, 2]
distance travelled: 2.738743322903475
Computing time: 0.00011777877807617188 seconds


Local search algorithm
-----------
best solution: [0, 5, 4, 8, 1, 9, 3, 6, 7, 2]
distance travelled: 2.738743322903475
Computing time: 0.00045490264892578125 seconds


Variable neighborhood search algorithm
-----------
greedy x: [0, 5, 4, 8, 1, 9, 3, 6, 7, 2]
neighborhood_1: x = [0 5 4 8 1 9 2 6 7 3]
neighborhood_1: x = [0 5 4 8 1 9 2 7 6 3]
best solution: [0 5 4 8 1 9 2 7 6 3]
distance travelled: 2.576814374650054
Computing time: 0.051043033599853516 seconds


Simulated annealing algorithm
-----------
greedy x: [0, 5, 4, 8, 1, 9, 3, 6, 7, 2], distance: 2.738743322903475
iter: 121, better path found: [7 3 6 5 0 4 8 1 9 2], distance: 2.6921437884971797
worse_accept_count: 0
best solution: [7 3 6 5 0 4 8 1 9 2]
distance travelled: 2.6921437884971797
Computing time: 0.012388944625854492 seconds


---